<a href="https://colab.research.google.com/github/dhars1n1/Thirukural-chatbot/blob/main/%E0%AE%A4%E0%AE%BF%E0%AE%B0%E0%AF%81%E0%AE%95%E0%AF%8D%E0%AE%95%E0%AF%81%E0%AE%B1%E0%AE%B3%E0%AF%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Our intention is to create a thirukural chatbot
2. To do so, we need to perform vector retrieval
3. to do vector retrival we need to represent our text data as embeddings using some transformer based model

LOAD JSON DATA

In [ ]:
import json

# Path to your JSON file
thirukkural_data = "thirukkural.json"

# Load JSON data from file
with open(thirukkural_data, "r", encoding="utf-8") as f:
    data = json.load(f)
print("Thirukural loaded successfully")

# Path to your JSON file
meta_file_path = "detail.json"

# Load JSON data from file
with open(thirukkural_data, "r", encoding="utf-8") as f:
    meta_data = json.load(f)
print("Meta information loaded successfully")
meta_data=meta_data['kural']
print(len(meta_data))

Thirukural loaded successfully
Meta information loaded successfully
1330


In [ ]:
kural_translation = []  # List to store kural translation

for kural_dict in data["kural"]:
    kural_text = f"{kural_dict['Translation']}"  #kural translations    }
    kural_translation.append(kural_text)
print(len(kural_translation))



1330


In [ ]:
print(type(kural_translation[0]))

<class 'str'>


4. we need the dimension and a distance function (dot prod possibly) to find the similarity between vectors  
5. namma implementation la enna panna porom naa pinecone ah api key vechu initialize pannanum

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

# Download model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get embeddings of sentences
embeddings = model.encode(kural_translation)

print(len(embeddings))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1330


In [ ]:
print(len(kural_translation))
print(len(embeddings[0]))

1330
384


colab la oru vector index create panni adhula en thirukural data va ellam vectors ah store panni veka porom

In [ ]:
!pip install pinecone-client

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

api_key = "8826f026-a047-467a-959c-31e9344822ab"

pc = Pinecone(api_key=api_key)
index_name = 'myindex'

# Check if the index exists
if index_name in pc.list_indexes().names():
    # Delete the existing index
    pc.delete_index(index_name)
if 'myindex' not in pc.list_indexes().names():
    pc.create_index( name='myindex', dimension=384,spec=ServerlessSpec(cloud='aws',region='us-east-1'))



In [ ]:
import random
import itertools

vector_dim=384
vector_count=1330

# Generate a list of strings from "1" to "1330"
ids = [str(i) for i in range(1, 1331)]


#function to handle chunking of pairs
def chunks(iterable,batch_size=100):
  it = iter(iterable)
  chunk = tuple(itertools.islice(it, batch_size))
  while chunk:
    yield chunk
    chunk = tuple(itertools.islice(it, batch_size))



In [ ]:
print(len(ids))

1330


7. adhuku aprom querying panni search panlam andha index kulla irukra vector form la irukra data vaa
9. aprom namma andha Kural and adhoda meta info vaa display panikalaam

In [ ]:
index = pc.Index('myindex', dimension=vector_dim)
# for i in range(len(ids)):

#   final_data = map(lambda id, embedding: (id, embedding), ids, embeddings)
#   final_data=map(final_data,meta_data[i])
#   final_data=list(final_data)

final_data = []
for i in range(len(ids)):
    vector = {"id": ids[i], "values": embeddings[i], "metadata": meta_data[i]}
    final_data.append(vector)
#print(final_data[1])
for chunk in chunks(final_data,batch_size=100):
  index.upsert(vectors=chunk)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1300}},
 'total_vector_count': 1300}

In [ ]:
import numpy as np

In [ ]:
# vector dimension 384 - convert the given word or sentence into a vector of dimension 384 for querying


# Example sentence
sentence ="im doubtful"

# Convert the sentence into a vector
sentence_vector = (model.encode(sentence)).tolist()


index.query(vector=sentence_vector, top_k=3, include_metadata=True)



{'matches': [{'id': '190',
              'metadata': {'Line1': 'ஏதிலார் குற்றம்போல் தங்குற்றங் '
                                    'காண்கிற்பின்',
                           'Line2': 'தீதுண்டோ மன்னும் உயிர்க்கு.',
                           'Number': 190.0,
                           'Translation': 'No harm would fall to any man  If '
                                          'each his own defect could scan',
                           'couplet': "If each his own, as neighbours' faults "
                                      'would scan,Could any evil hap to living '
                                      'man',
                           'explanation': 'If they observed their own faults '
                                          'as they observe the faults of '
                                          'others, would any evil happen to '
                                          'men ?',
                           'mk': 'பிறர் குற்றத்தைக் காண்பவர்கள் தமது '
                         